In [15]:
# 每一步的动作 负责把向量转化为方便运算的数据结构
# 皮带机和
from collections import namedtuple
import numpy as np
class PortCraneTask(
    namedtuple('PortCraneTask',
               'port_crane_index, weight, carbin_index, conveyor_system_index, conveyor_address_index'.split(', '))):
    def __init__(self, *args):
        # print(args)
        super().__init__()

    def __repr__(self):
        return 'PortCraneTask(weight={}, carbin_index={}, conveyor_system_index={}, conveyor_address_index={})'.format(
            self.weight, int(self.carbin_index), int(self.conveyor_system_index), int(self.conveyor_address_index))

    # def toTaskByTime(self, start_time):
    #     port_crane = PublicDataManager.port_crane_list[self.port_crane_index]
    #     return TaskByTime(PublicTool.generate_random_ud64(30), start_time,
    #                       start_time + self.weight / port_crane.get_carrying_capacity() * 60 * 60,
    #                       port_crane,
    #                       PublicDataManager.cargo_ship_list[0], int(self.carbin_index),
    #                       PublicDataManager.conveyor_list[int(self.conveyor_system_index)],
    #                       PublicDataManager.warehouse_quota_manager[int(self.conveyor_address_index)],
    #                       port_crane.get_carrying_capacity())

# 单步动作
class Action:
    def __init__(self,params_list):
        # print(params_list)
        self.port_crane_task_list = []
        index = 0
        params_list = [int(item) for item in params_list]
        for i in range(0,len(params_list),len(PortCraneTask._fields)-1):
            temp_list = [index, ] + params_list[i:i+len(PortCraneTask._fields)-1]
            temp_port_crane_task = PortCraneTask(*tuple(temp_list))
            self.port_crane_task_list.append(self.work_index(temp_port_crane_task))
            index += 1
    
    def __str__(self) -> str:
        return 'Action({})'.format(self.port_crane_task_list)
    
    def get_port_crane_task_list(self):
        return self.port_crane_task_list
    
    def get_weight_count(self):
        weight_count = 0
        for port_crane_task in self.port_crane_task_list:
            weight_count += port_crane_task.weight
        return weight_count
    
    def work_index(self,portCrane_task):
        # 处理索引 有的索引太大了
        conveyor_system_index = portCrane_task.conveyor_system_index
        carbin_index = portCrane_task.carbin_index
        if carbin_index>800:
            return PortCraneTask(portCrane_task.port_crane_index,portCrane_task.weight,8,portCrane_task.conveyor_system_index,portCrane_task.conveyor_address_index)
        return PortCraneTask(portCrane_task.port_crane_index,portCrane_task.weight,portCrane_task.carbin_index//100,portCrane_task.conveyor_system_index,portCrane_task.conveyor_address_index)
        
    def to_array(self):
        return np.array(self.port_crane_task_list)


a = Action([1,2,3,4,5,6,7,8,9,10,11,12,])
a.to_array()

array([[ 0,  1,  0,  3,  4],
       [ 1,  5,  0,  7,  8],
       [ 2,  9,  0, 11, 12]])

In [16]:
import numpy as np

class State:
    def __init__(self,params_list):
        self.cargo_ship_weight_list = []
        self.port_crane_carring_list = []
        self.from_array(params_list)

    def __str__(self) -> str:
        return 'State(cargo_ship_weight_list{},port_crane_carring_list{})'.format(self.cargo_ship_weight_list, self.port_crane_carring_list)
    
    def to_array(self):
        return np.array(self.cargo_ship_weight_list + self.port_crane_carring_list)
    
    def from_array(self,array):
        self.cargo_ship_weight_list = array[:9] # 假如是一个9编组的船
        self.port_crane_carring_list = array[9:]# 港机的运输效率

    def transite(self,action:Action):
        # 状态转移
        temp_time_list = []
        for task in action.get_port_crane_task_list():
            self.cargo_ship_weight_list[task.carbin_index] -= task.weight
            temp_time_list.append(task.weight/self.port_crane_carring_list[task.port_crane_index])
        return max(temp_time_list) * 60*60  # 返回使用的最长时间
    
    def is_empty(self):
        return all([item <=0 for item in self.cargo_ship_weight_list])
    
s = State([1,2,3,4,5,6,7,8,9,10,11,12,13,])
print(s)

State(cargo_ship_weight_list[1, 2, 3, 4, 5, 6, 7, 8, 9],port_crane_carring_list[10, 11, 12, 13])


In [17]:
from random import randint

#定义环境
class Environment:
    def __init__(self):
        self.state = None
        self.time_spend = 0 # 使用的总共时间 要惩罚时间的
        self.reset()
    def reset(self):
        self.state = State([randint(10000,30000) for i in range(9)] + [1500] * 3)
        return self.state

    def is_over(self):
        return self.state.is_empty()

    def get_state_reword(self,action):
        # 当前状态的分数,比如为负数了还有人来卸载就应该扣分
        weight = action.get_weight_count() #总共卸载的重量
        reword = weight/1000
        if self.is_over():
            # 惩罚时间
            reword += 1000 - self.time_spend /60/12
        return reword

    def step(self,action):
        time_ = self.state.transite(action)
        self.time_spend += time_
        reword = self.get_state_reword(action)
        return self.state,reword,self.is_over(),None
    

class MyWrapper:
    def __init__(self):
        self.env = Environment()
        self.step_n = 0

    def reset(self):
        state = self.env.reset()
        self.step_n = 0
        return state

    def step(self, action):
        state, reward, terminated, info = self.env.step(action)
        over = terminated
        #限制最大步数
        self.step_n += 1
        if self.step_n > 20:
            over = True

        return state, reward, over
env = MyWrapper()

env.reset()



In [18]:
import torch

#定义模型,评估状态下每个动作的价值
model = torch.nn.Sequential(
    torch.nn.Linear(12, 6*64),
    torch.nn.ReLU(),
    torch.nn.Linear(6*64, 6*64),
    torch.nn.ReLU(),
    torch.nn.Linear(6*64, 12),
    torch.nn.ReLU(),
)

model

Sequential(
  (0): Linear(in_features=12, out_features=384, bias=True)
  (1): ReLU()
  (2): Linear(in_features=384, out_features=384, bias=True)
  (3): ReLU()
  (4): Linear(in_features=384, out_features=12, bias=True)
  (5): ReLU()
)

In [19]:
from loguru import logger


#玩一局游戏并记录数据
def play(show=False):
    data = []
    reward_sum = 0

    state = env.reset()
    over = False
    while not over:
        
        action = model(torch.FloatTensor(state.to_array()).reshape(1, 12))
        # print(action)
        next_state, reward, over = env.step(Action(list(action.detach().numpy())[0]))

        data.append((state, action, reward, next_state, over))
        reward_sum += reward

        state = next_state

        if show:
            logger.info(f"state: {state}, action: {action}, reward: {reward}, over: {over}")

    return data, reward_sum


play()[-1]

64.08699999999999

In [20]:
import random

#数据池
class Pool:

    def __init__(self):
        self.pool = []

    def __len__(self):
        return len(self.pool)

    def __getitem__(self, i):
        return self.pool[i]

    #更新动作池 多玩一些游戏
    def update(self):
        #每次更新不少于N条新数据
        old_len = len(self.pool)
        while len(pool) - old_len < 200:
            self.pool.extend(play()[0])

        #只保留最新的N条数据
        self.pool = self.pool[-2_0000:]

    #获取一批数据样本 从数据池种获取一些数据
    def sample(self):
        data = random.sample(self.pool, 64)
        print('data[0][1]', data[0][1])
        print(torch.LongTensor(data[0][1]))
        state = torch.FloatTensor([i[0].to_array() for i in data]).reshape(-1, 12)
        action = torch.LongTensor([i[1] for i in data]).reshape(-1, 12)
        reward = torch.FloatTensor([i[2] for i in data]).reshape(-1, 1)
        next_state = torch.FloatTensor([i[3] for i in data]).reshape(-1, 12)
        over = torch.LongTensor([i[4] for i in data]).reshape(-1, 1)

        return state, action, reward, next_state, over


pool = Pool()
pool.update()
pool.sample()

len(pool), pool[0]

data[0][1] tensor([[1911.6669,   87.6308,    0.0000,    0.0000, 1297.3422, 1592.2814,
         1460.0532, 2403.9197,    0.0000,    0.0000, 1202.9396, 2914.6558]],
       grad_fn=<ReluBackward0>)


TypeError: only integer tensors of a single element can be converted to an index